In [3]:
import os
import aisdb
import pickle
import numpy as np
import pandas as pd
import nest_asyncio
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cartopy.feature as cfeature
from aisdb import DBConn, DBQuery, DomainFromPoints
from aisdb.database.dbconn import PostgresDBConn
from shapely.geometry import Point, MultiPoint
from aisdb.webdata.shore_dist import CoastDist
from aisdb.webdata.shore_dist import ShoreDist
from aisdb.webdata.shore_dist import PortDist
from datetime import datetime, timedelta
from aisdb.gis import DomainFromTxts
from aisdb.database import sqlfcn_callbacks
from shapely import prepare
from PIL import ImageFile
from tqdm import tqdm
nest_asyncio.apply()
ImageFile.LOAD_TRUNCATED_IMAGES = True
from aisdb.weather.era5 import ClimateDataStore # for weather
from aisdb.weather.weather_fetch import FetchClimateData

In [4]:

os.environ["CDSAPI_URL"] = "https://cds.climate.copernicus.eu/api/v2"
os.environ["CDSAPI_KEY"] = "7b09e7d6-1d93-4586-96be-0f31464d3b92"

In [7]:
# >>> PostgreSQL Information <<<
db_user='parth'            # DB User
db_dbname='aisviz'         # DB Schema
db_password='parth2025'    # DB Password
db_hostaddr='127.0.0.1'    # DB Host address

dbconn = PostgresDBConn(
    port=5588,             # PostgreSQL port
    user=db_user,          # PostgreSQL username
    dbname=db_dbname,      # PostgreSQL database
    host=db_hostaddr,      # PostgreSQL address
    password=db_password,  # PostgreSQL password
)

In [14]:
xmin, ymin, xmax, ymax = -70, 45, -58, 53
gulf_bbox = [xmin, xmax, ymin, ymax]
start_time = datetime(2023, 8, 1)
end_time = datetime(2023, 8, 30)

qry = DBQuery(
    dbconn=dbconn,
    start=start_time, end=end_time,
    xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax,
    callback=aisdb.database.sqlfcn_callbacks.in_time_bbox_validmmsi
)

ais_tracks = []
rowgen = qry.gen_qry()
tracks = aisdb.track_gen.TrackGen(rowgen, decimate="True")

In [15]:
params_provided = {
    "variable": "10m u-component of wind",
    "start_time": start_time,
    "end_time": end_time,
    "area": gulf_bbox,
}

fetcher = FetchClimateData(dataset="test", params_requested= params_provided, output_path="climate_data.grib")
fetcher.fetch_data()

NameError: name 'request' is not defined